# Imports


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

In [34]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules

# Load data

In [60]:
df = pd.read_csv("data/clicks_sample.csv", index_col=0)

In [61]:
df

,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
user_id,,,,,,,,,,,
0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2
...,...,...,...,...,...,...,...,...,...,...,...
705,1506828968165442,1506828968000,2,119592,1506830912301,4,1,17,1,21,2
705,1506828968165442,1506828968000,2,284847,1506830942301,4,1,17,1,21,2
706,1506828979881443,1506828979000,3,108854,1506829027334,4,3,2,1,25,1


In [52]:
articles_per_session = []
for session_id, data in df.groupby("session_id"):
    articles_per_session += [list(enumerate(data["click_article_id"].values))]

In [53]:
x = articles_per_session

In [55]:
x[:10]

[[(0, 157541), (1, 68866)],
 [(0, 235840), (1, 96663)],
 [(0, 119592), (1, 30970)],
 [(0, 236065), (1, 236294)],
 [(0, 48915), (1, 44488)],
 [(0, 168868), (1, 207122), (2, 286413)],
 [(0, 202436), (1, 288431)],
 [(0, 235840), (1, 284847)],
 [(0, 332114), (1, 284847), (2, 114161)],
 [(0, 284346), (1, 285412)]]

# Create frequent patterns

In [66]:
te = TransactionEncoder()
te_ary = te.fit(x).transform(x)
df2 = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df2, min_support=0.005, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.041018,"((1, 68866))"
1,0.026874,"((0, 157541))"
2,0.090523,"((1, 96663))"
3,0.084866,"((0, 235840))"
4,0.147100,"((0, 119592))"
...,...,...
110,0.008487,"((0, 68866), (1, 284847))"
111,0.005658,"((0, 108854), (1, 157541))"
112,0.007072,"((1, 168868), (2, 106984))"
113,0.005658,"((2, 145166), (1, 284847))"


In [71]:
df3 = association_rules(frequent_itemsets, metric="support", min_threshold=0.005)

In [72]:
df3['n'] = df3['support'] * len(df)

In [73]:
df3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,n
0,"((0, 108854))","((1, 68866))",0.131542,0.041018,0.009901,0.075269,1.835002,0.004505,1.037038,18.643564
1,"((1, 68866))","((0, 108854))",0.041018,0.131542,0.009901,0.241379,1.835002,0.004505,1.144786,18.643564
2,"((0, 119592))","((1, 68866))",0.147100,0.041018,0.005658,0.038462,0.937666,-0.000376,0.997341,10.653465
3,"((1, 68866))","((0, 119592))",0.041018,0.147100,0.005658,0.137931,0.937666,-0.000376,0.989364,10.653465
4,"((0, 96663))","((1, 68866))",0.091938,0.041018,0.005658,0.061538,1.500265,0.001887,1.021866,10.653465
...,...,...,...,...,...,...,...,...,...,...
79,"((2, 106984))","((1, 168868))",0.009901,0.021216,0.007072,0.714286,33.666667,0.006862,3.425743,13.316832
80,"((2, 145166))","((1, 284847))",0.025460,0.087694,0.005658,0.222222,2.534050,0.003425,1.172964,10.653465
81,"((1, 284847))","((2, 145166))",0.087694,0.025460,0.005658,0.064516,2.534050,0.003425,1.041750,10.653465
82,"((1, 202436))","((0, 119592))",0.014144,0.147100,0.008487,0.600000,4.078846,0.006406,2.132249,15.980198
